# 2. OpenAI の チャット API の基礎


## 2.3. 入出力の長さの制限や料金に影響する「トークン」


### トークン


In [1]:
!pip install tiktoken==0.7.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00


In [2]:
import tiktoken

text = "ChatGPT"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
for token in tokens:
    print(encoding.decode([token]))

Chat
GPT


### Tokenizer と tiktoken の紹介


In [3]:
import tiktoken

text = "LLMを使ってクールなものを作るのは簡単だが、プロダクションで使えるものを作るのは非常に難しい。"

encoding = tiktoken.encoding_for_model("gpt-4o")
tokens = encoding.encode(text)
print(len(tokens))

37


## 2.4. Chat Completions API を試す環境の準備


### OpenAI の API キーの準備


In [4]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 2.5. Chat Completions API のハンズオン


### OpenAI のライブラリ


#### 【注意】既知のエラーについて

openai パッケージが依存する httpx のアップデートにより、`openai==1.40.6` を使用する箇所で `TypeError: Client.__init__() got an unexpected keyword argument 'proxies'` というエラーが発生するようになりました。

このエラーは、`!pip install httpx==0.27.2` のように、httpx の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install httpx==0.27.2` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [6]:
!pip install openai==1.40.6 httpx==0.27.2

### Chat Completions API の呼び出し


In [8]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9n0LH2KcjoySQkfxVzCHSEPriyNN",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできてうれしいです。今日はどんなことをお話ししたいですか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741673025,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_06737a9306",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 25,
    "total_tokens": 55,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### 会話履歴を踏まえた応答を得る


In [9]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {"role": "assistant", "content": "こんにちは、ジョンさん！お会いできてうれしいです。今日はどんなことをお話ししたいですか？"},
        {"role": "user", "content": "私の名前が分かりますか？"},
    ],
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9n0dSdhLCqWH2IX1Ki7o3Ysid8ap",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前はジョンさんです！他に何かお手伝いできることがあれば教えてください。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741673043,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_06737a9306",
  "usage": {
    "completion_tokens": 30,
    "prompt_tokens": 71,
    "total_tokens": 101,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


### ストリーミングで応答を得る


In [10]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
    ],
    stream=True,
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？

### JSON モード


In [11]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)
print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


### Vision（画像入力）


In [13]:
from openai import OpenAI

client = OpenAI()

image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は、書籍の表紙を示しています。タイトルは「ChatGPT/LangChainによるチャットシステム構築【実践】入門」と記載されています。著者の名前が2人分書かれており、その下には「ChatGPT」という文字が大きく表示されています。また、表紙にはカラフルな鳥のイラストが描かれています。全体的に青い背景が使われております。


### （コラム）Completions API


In [15]:
from openai import OpenAI

client = OpenAI()

response = client.completions.create(
    model="gpt-3.5-turbo-instruct",
    prompt="こんにちは！私はジョンと言います！",
)
print(response.to_json(indent=2))

{
  "id": "cmpl-B9n6euZ5hXBE0BVyxGnp4afVpnggz",
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\n\n初めまして、ジョンさん！私はAIのチ"
    }
  ],
  "created": 1741673416,
  "model": "gpt-3.5-turbo-instruct:20230824-v2",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 11,
    "total_tokens": 27
  }
}


## 2.6. Function calling


### Function calling のサンプルコード


In [16]:
import json


def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [17]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [23]:
from openai import OpenAI

client = OpenAI()

messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B9nVUEuNTWQyEP2jfOO2Lcpi5S3bd",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_nUzV2mpYWMYOhDzWvA4TSuBP",
            "function": {
              "arguments": "{\"location\":\"Tokyo, JP\",\"unit\":\"celsius\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1741674956,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_f9f4fb6dbf",
  "usage": {
    "completion_tokens": 23,
    "prompt_tokens": 81,
    "total_tokens": 104,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,

In [26]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [34]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": "celsius"}


In [35]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_nUzV2mpYWMYOhDzWvA4TSuBP",
        "function": {
          "arguments": "{\"location\":\"Tokyo, JP\",\"unit\":\"celsius\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_nUzV2mpYWMYOhDzWvA4TSuBP",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": \"celsius\"}"
  }
]


In [36]:
second_response = client.chat.completions.create(
    model="gpt-4o",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-B9na0Z6qRoyyoydXAQUSoBPZMDQRQ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "現在の東京の天気は気温10℃です。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1741675236,
  "model": "gpt-4o-2024-08-06",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_0d4eb8a50b",
  "usage": {
    "completion_tokens": 14,
    "prompt_tokens": 66,
    "total_tokens": 80,
    "prompt_tokens_details": {
      "cached_tokens": 0,
      "audio_tokens": 0
    },
    "completion_tokens_details": {
      "reasoning_tokens": 0,
      "audio_tokens": 0,
      "accepted_prediction_tokens": 0,
      "rejected_prediction_tokens": 0
    }
  }
}


## tool_choice

> パラメータ「tool_choice」 Function callingに関連して、Chat Completions APIのリクエストには「tool_choice」というパラメータもあります。tool_choiceというパラメータに"none"を指定すると、LLMは関数を呼び出すような応答をせず、通常のテキストを返してきます。tool_choiceを"auto"に設定すると、LLMは入力に応じて、指定した関数を使うべきと判断した場合は関数名と引数を返すようになります。パラメータ「tool_choice」のデフォルトの動作は、toolsを与えなかった場合は"none"、toolsを与えた場合は"auto"となっています。 また、パラメータ「tool_choice」には「{"type": "function", "function": {"name": "<関数名>"}}」という値を指定することができます。このように関数名を指定すると、LLMに指定した関数を呼び出すことを強制させることができます。


西見 公宏; 吉田 真吾; 大嶋 勇樹. LangChainとLangGraphによるRAG・AIエージェント［実践］入門 エンジニア選書 (p. 110). (Function). Kindle Edition.